## Making restriction maps of the BACs in the RP11 library on chromosome 19

This jupyter notebook contains an example of running the pipeline, the input data is reduced to only BACs in RP11 on chromosome 19 so it should only take a few minutes to run any command.

Importing bacmapping as bmap to have access to the functions, importing plt from matplotlib so we can show a nice map when we're done, os is useful for moving around files

In [ ]:
import bacmapping as bmap
import matplotlib.pyplot as plt
import os

Setting some variables, accession number, library and chromosome size we're restricting the library to, chunksize and cpus related to how much memory and cpus to use in processing, and your email.

In [ ]:
email = "example@website.com" # Remember to give NCBI your email!

acc = 'NC_000019.10' # accession number of human genome chromosome 19
lib = 'RP11' # library we're interested in
chrom = '19' # chromosome number we're interested in
chunksize = 5000 # how many entries to read into memory in each chunk, bigger = faster and more memory usage
cpus = 8 # how many cpus to use in multiprocessing, more means faster but you need the cpus free

We need to get the clones we're going to map and download the sequence we are using. For the full pipeline, we'd run getNewClones instead of getNewClonesMiniset. This would make the mapping take longer and use more space.

This will make two new folders, details and sequences, details has all the information on the clones, sequences has the sequences

It will take a few minutes to download everything

In [ ]:
bmap.getNewClonesMiniset(email, lib, acc, chrom, chunk_size = chunksize) # a variation on downloading the dataset which only downloads one library and one chromosome

Map the end-sequenced BACs in the example dataset by running mapPlacedClones, this will take a few minutes to run

In [ ]:
bmap.mapPlacedClones(cpustouse=cpus, chunk_size=chunksize)

Some of the BACs are insert-sequenced and use a different function to be restriction mapped, mapSequencedClones

In [ ]:
bmap.mapSequencedClones(cpustouse=cpus)

The statistics on this dataset can be determined by running the following commands, which are detailed in the readme, they'll save 4 csv files with the results of this analysis

In [ ]:
bmap.countPlacedBACs()
bmap.getCoverage()
bmap.getAverageLength()
bmap.getSequencedClonesStats()

Output files:
- countPlacedBACs counts the number of BACs in each end-sequenced library and saves this to counts.csv
- getCoverage determines the number of bases per chromosome which are included in the inserts of end-sequenced BACs in each library and saves this to coverage.csv
- getAverageLength finds the average length of clones in each end-sequenced library and saves this to averagelength.csv
- getSequencedClonesStats gets both the average length and number of clones for each library of insert-sequenced clones and saves this to sequencedStats.csv

Finally, let's explore one set of maps produced in the library, we'll return all the maps for one BAC which is included in the library and then get an image of the produced map.

In [ ]:
name = 'RP11-793A20'
enzyme = 'SgrAI'
maps = bmap.getMaps(name)
#print(maps) #this is a big dataframe of all the maps, uncomment to check it out
rmap = bmap.getRestrictionMap(name,enzyme)
print('Sites in ' + name + ' where ' + enzyme + ' cuts: '+ str(rmap))
plt = bmap.drawMap(name, enzyme)
plt.show()

-   maps from bmap.getMaps(name) is a series of all the restriction maps for RP11-793A20
-   rmap from bmap.getRestrictionMap(name,enzyme) is just the cut locations of FspI in RP11-793A20
-   plt is a visual representation of rmap


Then let's find pairs that include our BAC of interest. We'll set longestoverlap, the longest acceptable overlap in the overlapping end, to 500 and shortestoverlap, the shortest acceptable overlap, to 0. This means that we'll also include BACs which are linearized at the same site. 

In [ ]:
name = 'RP11-793A20'
pairs = (bmap.findPairsFromName(name,longestoverlap=500,shortestoverlap=0))
pairs

Instead of just finding one, we can find all BAC pairs which are linearized to produce overlapping ends. This code will make a folder named pairs with each chromosome detailing all the possible pairs of BACs.

In [ ]:
bmap.makePairs(cpustouse=cpus,longestoverlap=500,shortestoverlap=0)

## Functions to explore the library

### getMaps

Given the name of a BAC, returns a dataframe containing all the restriction maps related to that BAC.

In [ ]:
name = 'RP11-1053G2'
maps = bmap.getMaps(name)
maps

### getRestrictionMap

Given the name of a BAC and an enzyme, returns the cut locations.

In [ ]:
name = 'RP11-1053G2'
enzyme = 'FspI'
rmap = bmap.getRestrictionMap(name, enzyme)
rmap

### getRightIsoschizomer

Given an enzyme name, returns the enzyme name and Bio.restriction class which corresponds to the isoschizomer which is in the database. Name is a string of the enzyme name, libraryenzyme is the Bio.restriction class of the enzyme.

In [ ]:
testenzyme = "BsaI"
name, libraryenzyme = bmap.getRightIsoschizomer(testenzyme)
print(name)

### DrawMap

Draws a map for a given BAC and enzyme.

In [ ]:
name = 'RP11-1053G2'
enzyme = 'FspI'
rmap = bmap.drawMap(name, enzyme)

### getSequenceFromName

Given the name of a BAC, tries to return the sequence of that insert.

In [ ]:
name = 'RP11-1055H23' # an end-sequenced clone
#name = 'RP11-618P17' # an insert-sequenced clone 
seq = bmap.getSequenceFromName(name)
print(seq)

### getSequenceFromLoc

Given a chromosome, start and end location, returns sequence of that location.

In [ ]:
chrom = 19
start = 100000
end = 500000
seq = bmap.getSequenceFromLoc(chrom,start,end)
print(seq)

### getMapsFromLoc

Given a chromosome, start and end location, returns all the maps in that region.

In [ ]:
chrom = 19
start = 100000
end = 500000
maps = bmap.getMapsFromLoc(chrom,start,end)
maps

### findOverlappingBACs

Given a BAC name, returns a dataframe with details for all the BACs which overlap the BAC.

In [ ]:
name = 'RP11-1053G2'
bacs = bmap.findOverlappingBACs(name)
bacs